In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

class PCA:
    def __init__(self, target_explained_variance=None):
        """
        Initialize the PCA object with a target level of explained variance.
        :param target_explained_variance: float, the target level of explained variance
        """
        self.target_explained_variance = target_explained_variance
        self.feature_size = -1

    def standardize(self, X):
        """
        Standardize the features using sklearn's StandardScaler.
        :param X: Input data with shape (m, n) where m is the number of observations and n is the number of features
        :return: Standardized features
        """
        return StandardScaler().fit_transform(X)

    def compute_mean_vector(self, X_std):
        """
        Compute the mean vector of the standardized data.
        :param X_std: Standardized data
        :return: Mean vector with shape (n,)
        """
        return np.mean(X_std, axis=0)

    def compute_cov(self, X_std, mean_vec):
        """
        Compute the covariance matrix using the mean vector.
        :param X_std: Standardized data
        :param mean_vec: Mean vector
        :return: Covariance matrix with shape (n, n)
        """
        m = X_std.shape[0]
        return (X_std - mean_vec).T.dot(X_std - mean_vec) / (m - 1)

    def compute_eigen_vector(self, cov_mat):
        """
        Compute eigenvectors and eigenvalues using numpy's eigenvalue function.
        :param cov_mat: Covariance matrix
        :return: Tuple containing eigenvalues and eigenvectors
        """
        eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)
        return eigen_vals, eigen_vecs

    def compute_explained_variance(self, eigen_vals):
        """
        Sort eigenvalues and compute explained variance.
        Explained variance informs the amount of information (variance) that can be attributed to each principal component.
        :param eigen_vals: Eigenvalues
        :return: Explained variance
        """
        total = sum(eigen_vals)
        var_exp = [(i / total) for i in sorted(eigen_vals, reverse=True)]
        return np.array(var_exp)

    def cumulative_sum(self, var_exp):
        """
        Compute the cumulative sum of the explained variance.
        :param var_exp: Explained variance
        :return: Cumulative explained variance
        """
        return np.cumsum(var_exp)

    def compute_weight_matrix(self, eig_pairs, cum_var_exp):
        """
        Compute the weight matrix of top principal components based on the target explained variance.
        :param eig_pairs: List of tuples containing eigenvalues and eigenvectors, sorted by eigenvalues in descending order
        :param cum_var_exp: Cumulative explained variance
        :return: Weight matrix with shape (n, k)
        """
        k = np.argmax(cum_var_exp >= self.target_explained_variance) + 1
        return np.hstack([eig_pairs[i][1].reshape(-1, 1) for i in range(k)])

    def transform_data(self, X_std, matrix_w):
        """
        Transform data to a subspace using the weight matrix.
        :param X_std: Standardized data
        :param matrix_w: Weight matrix
        :return: Transformed data in the subspace
        """
        return X_std.dot(matrix_w)

    def fit(self, X):
        """
        Fit the model to the data and return the transformed features.
        The fit method standardizes the data, computes the weight matrix, and then transforms the data.
        The number of dimensions (k) is determined based on the target explained variance.
        :param X: Input data with shape (m, n)
        :return: Transformed data with shape (m, k)
        """
        self.feature_size = X.shape[1]

        # Standardize the data
        X_std = self.standardize(X)

        # Compute the mean vector and covariance matrix
        mean_vec = self.compute_mean_vector(X_std)
        cov_mat = self.compute_cov(X_std, mean_vec)

        # Compute eigenvalues and eigenvectors
        eigen_vals, eigen_vecs = self.compute_eigen_vector(cov_mat)
        eig_pairs = [(eigen_vals[i], eigen_vecs[:, i]) for i in range(len(eigen_vals))]
        eig_pairs = sorted(eig_pairs, key=lambda x: x[0], reverse=True)

        # Compute explained variance and cumulative explained variance
        var_exp = self.compute_explained_variance(eigen_vals)
        cum_var_exp = self.cumulative_sum(var_exp)

        # Compute the weight matrix based on the target explained variance
        matrix_w = self.compute_weight_matrix(eig_pairs, cum_var_exp)

        # Transform the data to the new subspace
        return self.transform_data(X_std, matrix_w)